In [1]:
#SETTINGS and imports

from settings.settings import data_settings

import pandas as pd
import numpy as np
import tensorflow as tf
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 200


In [2]:
# LOADING DATA
# df_with_target = pd.read_csv(data_settings.data_with_target_path)
# df_with_indicators = pd.read_csv(data_settings.data_with_indicators_path)
df_final_not_cleaned = pd.read_csv(data_settings.data_final_not_cleaned_path)
df_final_not_cleaned

,Open,High,Low,Close,TickVol,Spread,ATR,buy_1,buy_2,buy_4,...,AROON_UP,EMA_INDICATOR_12,EMA_INDICATOR_26,EMA_INDICATOR_50,EMA_INDICATOR_100,SMA_INDICATOR_12,SMA_INDICATOR_26,SMA_INDICATOR_50,SMA_INDICATOR_100,DateTime
0,1.20088,1.20088,1.20007,1.20023,16,37,0.000447,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-02 00:00:00
1,1.20023,1.20092,1.20023,1.20092,24,29,0.000464,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-02 00:05:00
2,1.20093,1.20094,1.20075,1.20090,17,18,0.000445,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-02 00:10:00
3,1.20090,1.20090,1.20034,1.20058,8,26,0.000453,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-02 00:15:00
4,1.20048,1.20061,1.20048,1.20061,15,32,0.000430,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-02 00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327982,1.07281,1.07288,1.07277,1.07279,98,4,0.000266,0,0,0,...,92.0,1.072529,1.072279,1.072005,1.071519,1.072491,1.072275,1.071817,1.071493,2022-05-26 23:20:00
327983,1.07279,1.07302,1.07278,1.07291,104,4,0.000264,0,0,0,...,100.0,1.072588,1.072326,1.072041,1.071547,1.072570,1.072315,1.071827,1.071520,2022-05-26 23:25:00
327984,1.07291,1.07307,1.07286,1.07301,127,8,0.000260,0,0,0,...,100.0,1.072653,1.072377,1.072079,1.071576,1.072617,1.072348,1.071848,1.071554,2022-05-26 23:30:00
327985,1.07301,1.07310,1.07295,1.07303,56,8,0.000252,0,0,0,...,100.0,1.072711,1.072425,1.072116,1.071604,1.072677,1.072383,1.071872,1.071591,2022-05-26 23:35:00


In [3]:
# MAKING A READY DF
df = df_final_not_cleaned.copy()

# df.pop(df.PSAR_DOWN.name)
# df.pop(df.PSAR_UP.name)
# df.pop(df.KAMA.name)
df.pop(df.AVERAGE_TRUE_RANGE.name)
df=df.dropna().reset_index(drop=True)

date_time = pd.to_datetime(df.pop("DateTime"),format=data_settings.date_time_format)
timestamps = date_time.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day
df["Day_sin"] = (np.sin(timestamps * (2 * np.pi / day))+1)/2
df["Day_cos"] = (np.cos(timestamps * (2 * np.pi / day))+1)/2
df["Year_sin"] = (np.sin(timestamps * (2 * np.pi / year))+1)/2
df["Year_cos"] = (np.cos(timestamps * (2 * np.pi / year))+1)/2

# plt.plot(np.array(df["Day_sin"])[:1000])
# plt.plot(np.array(df["Day_cos"])[:1000])
# plt.plot(np.array(df["Year_sin"])[:1000])
# plt.plot(np.array(df["Year_cos"])[:1000])

#make targets
df.pop(df.buy_1.name)
df.pop(df.buy_2.name)
df.pop(df.buy_8.name)
df.pop(df.buy_16.name)

df.pop(df.sell_1.name)
df.pop(df.sell_2.name)
df.pop(df.sell_8.name)
df.pop(df.sell_16.name)
df.pop(df.windowsizes.name)
df.pop(df.do_nothing.name)

do_nothings = []
for i in range(len(df)):
    if(df.buy_4.iloc[i]==0 and df.sell_4.iloc[i]==0):
        do_nothings.append(1)
    else:
        do_nothings.append(0)
        
df["do_nothing"]= do_nothings
df["DateTime"]= date_time

df.to_csv(data_settings.data_final_path,index=False)

In [4]:
# Split to train and test

n=len(df)
train_df = df[:int((0.85)*n)].reset_index(drop=True)
test_df = df[int((0.85)*n):].reset_index(drop=True)
train_df.to_csv(data_settings.data_train_path,index=False)
test_df.to_csv(data_settings.data_test_path,index=False)

In [ ]:
# train = pd.read_csv(data_settings.data_train_path)
# test = pd.read_csv(data_settings.data_test_path)

In [ ]:
train_df = pd.read_csv(data_settings.data_train_path)
test_df = pd.read_csv(data_settings.data_test_path)
window_size = data_settings.window_size

test_buy = test_df.pop(test_df.buy_4.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.sell_4.name)

train_buy = train_df.pop(train_df.buy_4.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.sell_4.name)

In [ ]:
test_labels = (
                test_buy.to_numpy(),
                test_nothing.to_numpy(),
                test_sell.to_numpy()
               )
test_labels = np.array(test_labels)
test_labels = test_labels.transpose()
test_labels = test_labels.astype("float32")

train_labels = (
                train_buy.to_numpy(),
                train_nothing.to_numpy(),
                train_sell.to_numpy()
               )
train_labels = np.array(train_labels)
train_labels = train_labels.transpose()
train_labels = train_labels.astype("float32")

In [ ]:
def x_train_generator():
    for i in range(window_size,len(train_df)):
        window=train_df[i-window_size:i]
        yield ((window-window.min())/(window.max()-window.min())).to_numpy()

def x_test_generator():
    for i in range(window_size,len(test_df)):
        window=test_df[i-window_size:i]
        yield ((window-window.min())/(window.max()-window.min())).to_numpy()    
        
def y_train_generator():
    for i in range(window_size-1,train_labels.shape[0]):
        yield train_labels[i]

def y_test_generator():
    for i in range(window_size-1,test_labels.shape[0]):
        yield test_labels[i]
        
x_train_ds = tf.data.Dataset.from_generator(x_train_generator
                                        output_signature=(
                                            tf.TensorSpec(shape=(256,101), dtype=tf.float64)
                                            )
                                        ).batch(256)

x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
                                        output_signature=(
                                            tf.TensorSpec(shape=(256,101), dtype=tf.float64)
                                            )
                                        ).batch(256)
y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
                                        output_signature=(
                                            tf.TensorSpec(shape=(3), dtype=tf.float32)
                                            )
                                        ).batch(256)
y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
                                        output_signature=(
                                            tf.TensorSpec(shape=(3), dtype=tf.float32)
                                            )
                                        ).batch(256)

train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

In [ ]:
def money(initial,percent,month):
    
    if month ==0 :
        return initial
    
    else:
        return percent * money(initial,percent,month-1) + money(initial,percent,month-1)
    
